In [1]:
import geopandas as gpd
import pandas as pd
import random
url="https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/alberi-manutenzioni/exports/geojson?lang=en&timezone=Europe%2FRome"


In [2]:
alberi_geojson = gpd.read_file(url)

In [3]:
del alberi_geojson['geo_point_2d']
del alberi_geojson['zona_di_prossimita']
del alberi_geojson['area_statistica']
del alberi_geojson['quartiere']
del alberi_geojson['num_pt']
del alberi_geojson['isolato']
del alberi_geojson['campagna']

In [4]:
alberi_geojson.columns

Index(['cod_alb', 'classe', 'cl_h', 'dimora', 'irriga', 'pregio',
       'classe_circonferenza_diametro', 'data_inv', 'data_agg', 'd_edif',
       'data_impnt', 'in_patrim', 'anni_impnt', 'geometry'],
      dtype='object')

In [5]:
alberi_geojson = alberi_geojson.rename(columns={
    'cod_alb': 'id'
})
alberi_geojson = alberi_geojson.dropna(subset=['geometry'])
alberi_geojson = alberi_geojson[alberi_geojson['geometry'].notnull()]
alberi_geojson = alberi_geojson[alberi_geojson['geometry'].apply(lambda x: x.is_valid)]
alberi_geojson = alberi_geojson[alberi_geojson['geometry'].apply(lambda x: x.geom_type == 'Point')]

In [6]:
def getheight(x):
    h = 0
    if x is not None:
        if x.find(" - ") > -1:
            x= x.split(": ")
            x = x[1].split(" - ")
            x1 = x[0].replace("mt","")
            x2 = x[1].replace("mt","")
            h = random.randint(int(x1), int(x2))
        else:
            if x != "NULL":
                x = x.split(": ")
                x = x[1].replace("mt","").replace("<","").replace(">","")
                h = int(x)
    return h

In [7]:
alberi_geojson['height'] = alberi_geojson['cl_h'].apply(lambda x: getheight(x))

In [8]:
def getgirth(x):
    h = 0
    if x is not None:
        x = x.replace("–","-")
        if x.find("(") > -1:
            lx = x.split(" (")[1]
            if (lx.find(">") > -1):
                h = int(lx.split(" ")[1])
            elif (lx.find("<") > -1):
                print(x)
                h = int(lx.split(" ")[1]) / 100
            else:
                if lx.find("-") > -1:
                    x1= lx.split("-")[0]
                    x2 = lx.split("-")[1].replace("cm)","")
                    h = random.randint(int(x1), int(x2))  / 100
                else:
                    h = int(lx.split(" ")[0])  / 100
    return h

In [9]:
alberi_geojson['girth'] = alberi_geojson.classe_circonferenza_diametro.apply(lambda x: getgirth(x))

In [10]:
del alberi_geojson['cl_h']
del alberi_geojson['classe_circonferenza_diametro']

In [12]:
specie = [
    'Celtis australis', 'Platanus acerifolia', 'Fraxinus angustifolia',
    'Aesculus hippocastanum', 'Cedrus deodara', 'Quercus pubescens',
    'Quercus robur', 'Fraxinus excelsior', 'Prunus cerasifera (var. Pissardi)', 'Acer campestre',
    'Cercis siliquastrum', 'Ulmus americana', 'Tilia intermedia',
    'Fraxinus intermedia', "Populus nigra 'Italica'", 'Carpinus betulus',
    'Robinia pseudoacacia', 'Pinus pinea', 'Crataegus sspp', 'Celtis rugosa (Trema rugosa)',
    'Fraxinus ornus', 'Cedrus atlantica', 'Acer negundo', 'Tilia platyphyllos',
    'Ulmus pumila (siberiano)', 'Ulmus carpinifolia (campestre)', 'Quercus ilex',
    'Cupressus sempervirens', 'Prunus domestica', 'Acer platanoides',
    'Melia azedarach', 'Tilia cordata', 'Ligustrum japonicum', 'Fraxinus oxycarpa',
    'Ulmus sp', 'Prunus subhirtella', 'Acer pseudoplatanus',
    'Magnolia x soulangiana', 'Morus nigra', 'Populus alba',
    'Prunus spinosa', 'Liquidambar styraciflua', 'Taxus baccata',
    'Pinus nigra', 'Gleditschia triacanthos', 'Albizia julibrissin',
    'Pyrus calleriana “chanticlaire”', 'Populus alba piramidalis',
    'Zelkova sspp', 'Lagestroemia indica'
]

In [13]:
specie2 = list(alberi_geojson.classe.unique())

In [14]:
# Trova le specie presenti in specie ma non in specie2
specie_non_in_specie2 = [s for s in specie if s not in specie2]

# Trova le specie presenti in specie2 ma non in specie
specie2_non_in_specie = [s for s in specie2 if s not in specie]

print("Specie in 'specie' ma non in 'specie2':", specie_non_in_specie2)
print("Specie in 'specie2' ma non in 'specie':", specie2_non_in_specie)

Specie in 'specie' ma non in 'specie2': []
Specie in 'specie2' ma non in 'specie': ["Carpinus betulus 'Fastigiata'", "Photinia x fraseri 'Red Robin'", 'Cupressus macrocarpa', 'Pyrus calleriana', 'Morus alba', 'Prunus avium', None, 'Sophora japonica', 'Juglans regia', 'Populus canescens', 'Populus nigra', 'Populus hybrida', 'Ligustrum sp', 'Pyrus communis', 'Cupressus arizonica', 'Gingko biloba', 'Aesculus x carnea', "Prunus serrulata 'Kanzan'", 'Zelkova serrata', 'Acer freemanii', 'Liriodendron tulipifera', 'Populus x euroamericana', 'Eriobotrya japonica', 'Koelreuteria paniculata', 'Catalpa bignoniodes', 'Tilia sspp', 'Eucaliptus gunnii', 'Acer saccharinum', 'Ulmus laevis (olmo bianco o olmo ciliato)', 'Crataegus monogyna', "Gleditschia triacanthos 'Sunburst'", 'Quercus palustris', 'Prunus padus', 'Corylus colurna', 'Prunus laurocerasus', 'Hibiscus syriacus', 'Eleagnus angustifolia', 'Fraxinus americana', "Morus alba 'Pendula'", 'Acer opalus', 'Ailanthus altissima', 'Broussonetia papy

In [ ]:
specie = list(alberi_geojson.classe.unique())
# Creiamo un dataframe per associare nome comune e famiglia botanica.
# In questa fase iniziale, usiamo un dizionario di mapping parziale.
# Questo può essere completato in una seconda fase.

# Mapping semplificato (nome comune e famiglia botanica)
mapping = {
    'Celtis australis': ('Bagolaro', 'Cannabaceae'),
    'Platanus acerifolia': ('Platano comune', 'Platanaceae'),
    'Fraxinus angustifolia': ('Frassino meridionale', 'Oleaceae'),
    'Aesculus hippocastanum': ('Ippocastano', 'Sapindaceae'),
    'Cedrus deodara': ('Cedro dell’Himalaya', 'Pinaceae'),
    'Quercus pubescens': ('Rovere', 'Fagaceae'),
    'Quercus robur': ('Farnia', 'Fagaceae'),
    'Fraxinus excelsior': ('Frassino maggiore', 'Oleaceae'),
    'Prunus cerasifera (var. Pissardi)': ('Mirabolano rosso', 'Rosaceae'),
    'Acer campestre': ('Acero campestre', 'Sapindaceae'),
    'Cercis siliquastrum': ('Albero di Giuda', 'Fabaceae'),
    'Ulmus americana': ('Olmo americano', 'Ulmaceae'),
    'Tilia intermedia': ('Tiglio ibrido', 'Malvaceae'),
    'Fraxinus intermedia': ('Frassino intermedio', 'Oleaceae'),
    "Populus nigra 'Italica'": ('Pioppo cipressino', 'Salicaceae'),
    'Carpinus betulus': ('Carpino bianco', 'Betulaceae'),
    'Robinia pseudoacacia': ('Robinìa', 'Fabaceae'),
    'Pinus pinea': ('Pino domestico', 'Pinaceae'),
    'Crataegus sspp': ('Biancospino', 'Rosaceae'),
    'Celtis rugosa (Trema rugosa)': ('Bagolaro rugoso', 'Cannabaceae'),
    'Fraxinus ornus': ('Orniello', 'Oleaceae'),
    'Cedrus atlantica': ('Cedro dell’Atlante', 'Pinaceae'),
    'Acer negundo': ('Acero americano', 'Sapindaceae'),
    'Tilia platyphyllos': ('Tiglio a grandi foglie', 'Malvaceae'),
    'Ulmus pumila (siberiano)': ('Olmo siberiano', 'Ulmaceae'),
    'Ulmus carpinifolia (campestre)': ('Olmo campestre', 'Ulmaceae'),
    'Quercus ilex': ('Leccio', 'Fagaceae'),
    'Cupressus sempervirens': ('Cipresso comune', 'Cupressaceae'),
    'Prunus domestica': ('Susino', 'Rosaceae'),
    'Acer platanoides': ('Acero riccio', 'Sapindaceae'),
    'Melia azedarach': ('Albero dei rosari', 'Meliaceae'),
    'Tilia cordata': ('Tiglio selvatico', 'Malvaceae'),
    'Ligustrum japonicum': ('Ligustro giapponese', 'Oleaceae'),
    'Fraxinus oxycarpa': ('Frassino ossicarpo', 'Oleaceae'),
    'Ulmus sp': ('Olmo', 'Ulmaceae'),
    'Prunus subhirtella': ('Ciliegio invernale', 'Rosaceae'),
    'Acer pseudoplatanus': ('Acero di monte', 'Sapindaceae'),
    'Magnolia x soulangiana': ('Magnolia di Soulange', 'Magnoliaceae'),
    'Morus nigra': ('Gelso nero', 'Moraceae'),
    'Populus alba': ('Pioppo bianco', 'Salicaceae'),
    'Prunus spinosa': ('Prugnolo', 'Rosaceae'),
    'Liquidambar styraciflua': ('Liquidambar', 'Altingiaceae'),
    'Taxus baccata': ('Tasso', 'Taxaceae'),
    'Pinus nigra': ('Pino nero', 'Pinaceae'),
    'Gleditschia triacanthos': ('Gleditsia', 'Fabaceae'),
    'Albizia julibrissin': ('Albero della seta', 'Fabaceae'),
    'Pyrus calleriana “chanticlaire”': ('Pero ornamentale Chanticlaire', 'Rosaceae'),
    'Populus alba piramidalis': ('Pioppo bianco colonnare', 'Salicaceae'),
    'Zelkova sspp': ('Zelkova', 'Ulmaceae'),
    'Lagestroemia indica': ('Lagerstroemia', 'Lythraceae'),
    "Carpinus betulus 'Fastigiata'": ('Carpino fastigiato', 'Betulaceae'),
    "Photinia x fraseri 'Red Robin'":  ('Photinia rossa', 'Rosaceae'),
    'Cupressus macrocarpa':  ('Cipresso di Monterey', 'Cupressaceae'),
    'Pyrus calleriana': ('Pero ornamentale', 'Rosaceae'),
    'Morus alba': ('Gelso bianco', 'Moraceae'),
    'Prunus avium': ('Ciliegio dolce', 'Rosaceae'),
    'Sophora japonica':  ('Sofora del Giappone', 'Fabaceae'),
    'Juglans regia': ('Noce comune', 'Juglandaceae'),
    'Populus canescens': ('Pioppo grigio', 'Salicaceae'),
    'Populus nigra': ('Pioppo nero', 'Salicaceae'),
    'Populus hybrida': ('Pioppo ibrido', 'Salicaceae'), 
    'Ligustrum sp': ('Ligustro', 'Oleaceae'),
    'Pyrus communis': ('Pero domestico', 'Rosaceae'),
    'Cupressus arizonica': ('Cipresso dell\'Arizona', 'Cupressaceae'),
    'Gingko biloba': ('Ginkgo', 'Ginkgoaceae'),
    'Aesculus x carnea': ('Ippocastano rosso', 'Sapindaceae'), 
    "Prunus serrulata 'Kanzan'": ('Ciliegio ornamentale Kanzan', 'Rosaceae'),
    'Zelkova serrata': ('Zelkova del Giappone', 'Ulmaceae'), 
    'Acer freemanii': ('Acero di Freeman', 'Sapindaceae'), 
    'Liriodendron tulipifera': ('Albero dei tulipani', 'Magnoliaceae'), 
    'Populus x euroamericana': ('Pioppo euroamericano', 'Salicaceae'),
    'Eriobotrya japonica': ('Nespolo del Giappone', 'Rosaceae'),
    'Koelreuteria paniculata': ('Albero della pioggia dorata', 'Sapindaceae'),
    'Catalpa bignoniodes': ('Catalpa comune', 'Bignoniaceae'),
    'Tilia sspp': ('Tiglio', 'Malvaceae'),
    'Eucaliptus gunnii': ('Eucalipto del sidro', 'Myrtaceae'),
    'Acer saccharinum': ('Acero argentato', 'Sapindaceae'),
    'Ulmus laevis (olmo bianco o olmo ciliato)': ('Olmo bianco', 'Ulmaceae'),
    'Crataegus monogyna': ('Biancospino comune', 'Rosaceae'),
    "Gleditschia triacanthos 'Sunburst'": ('Gleditsia Sunburst', 'Fabaceae'),
    'Quercus palustris': ('Quercia delle paludi', 'Fagaceae'),
    'Prunus padus': ('Ciliegio a grappoli', 'Rosaceae'),
    'Corylus colurna': ('Nocciolo di Costantinopoli', 'Betulaceae'),
    'Prunus laurocerasus': ('Lauroceraso', 'Rosaceae'),
    'Hibiscus syriacus': ('Ibisco siriaco', 'Malvaceae'),
    'Eleagnus angustifolia': ('Olivo di Boemia', 'Elaeagnaceae'),
    'Fraxinus americana': ('Frassino bianco', 'Oleaceae'),
    "Morus alba 'Pendula'": ('Gelso bianco pendulo', 'Moraceae'),
    'Acer opalus': ('Acero opalo', 'Sapindaceae'),
    'Ailanthus altissima': ('Ailanto', 'Simaroubaceae'),
    'Broussonetia papyrifera': ('Gelso da carta', 'Moraceae'),
    'Morus sspp': ('Gelso', 'Moraceae'),
    'Juglans nigra': ('Noce nero', 'Juglandaceae'),
    'Malus sspp': ('Melo', 'Rosaceae'),
    'Olea europea': ('Olivo', 'Oleaceae'),
    'Quercus castaneifolia': ('Farnia castagnifoglia', 'Fagaceae'),
    'Prunus cerasifera': ('Mirabolano', 'Rosaceae'),
    'Libocedrus decurrens': ('Calocedro', 'Cupressaceae'),
    'Prunus dulcis, amygdalus': ('Mandorlo', 'Rosaceae'),
    'Populus sp.': ('Pioppo', 'Salicaceae'),    
    'Punica granatum': ('Melograno', 'Lythraceae'),
    'Ficus carica': ('Fico', 'Moraceae'),
    'Salix alba': ('Salice bianco', 'Salicaceae'),
    'Cedrus sp': ('Cedro', 'Pinaceae'),
    'Crataegus oxycantha': ('Biancospino', 'Rosaceae'),
    'Diospyros kaki': ('Diospiro / Kaki', 'Ebenaceae'),
    'Fraxinus oxycarpa Raywood': ('Frassino ossicarpo Raywood', 'Oleaceae'),
    'Sorbus aucuparia': ('Sorbo degli uccellatori', 'Rosaceae'),
    'Thuia orientalis': ('Tuia orientale', 'Cupressaceae'),
    "Populus alba 'Bolleana'": ('Pioppo bianco colonnare', 'Salicaceae'),
    'Prunus sp': ('Ciliegio / Prugno', 'Rosaceae'),
    'Taxodium distichum': ('Cipresso calvo', 'Cupressaceae'),
    'Prunus armeniaca': ('Albicocco', 'Rosaceae'),
    'Malus floribunda': ('Melo giapponese a fioritura abbondante', 'Rosaceae'),
    "Quercus robur 'Fastigiata'": ('Farnia fastigiata', 'Fagaceae'),
    'Magnolia grandiflora': ('Magnolia sempreverde', 'Magnoliaceae'),
    'Acer sp': ('Acero', 'Sapindaceae'),
    'Populus tremula': ('Pioppo tremulo', 'Salicaceae'),
    'Malus profusion': ('Melo ornamentale Profusion', 'Rosaceae'),
    'Tilia tomentosa': ('Tiglio argentato', 'Malvaceae'),
    'Prunus sargentii': ('Ciliegio di Sargent', 'Rosaceae'),
    "Parrotia persica 'Vanessa'": ('Albero di ferro Vanessa', 'Hamamelidaceae'),
    'Paulownia tomentosa': ('Paulonia', 'Paulowniaceae'),
    'Metasequoia sspp': ('Metasequoia', 'Cupressaceae'),
    'Prunus persica': ('Pesco', 'Rosaceae'),
    'Parrottia persica': ('Albero di ferro', 'Hamamelidaceae'),
    'Alnus glutinosa': ('Ontano nero', 'Betulaceae'),
    'Acer monspessulanum': ('Acero trilobo', 'Sapindaceae'),
    'Malus domestica': ('Melo domestico', 'Rosaceae'),
    "Prunus avium 'Plena'": ('Ciliegio doppio', 'Rosaceae'),
    'Fraxinus sp': ('Frassino', 'Oleaceae'),
    'Quercus rubra': ('Quercia rossa', 'Fagaceae'),
    'Ligustrum ovalifolium': ('Ligustro a foglia ovale', 'Oleaceae'),
    'Laurus nobilis': ('Alloro', 'Lauraceae'),
    'Ligustrum lucidum': ('Ligustro lucido', 'Oleaceae'),
    'Sambucus nigra': ('Sambuco nero', 'Adoxaceae'),
    "Cedrus atlantica 'Glauca'": ('Cedro dell’Atlante glauco', 'Pinaceae'),
    "Salix matsudana 'Tortuosa'": ('Salice tortuoso', 'Salicaceae'),
    'Prunus sargentii var. «\xa0Yedoensis\xa0»': ('Ciliegio giapponese', 'Rosaceae'),
    'Picea abies': ('Abete rosso', 'Pinaceae'),
    "Taxus baccata 'Fastigiata'": ('Tasso fastigiato', 'Taxaceae'),
    'Ostrya carpinifolia': ('Carpino nero', 'Betulaceae'),
    'Quercus sp': ('Quercia', 'Fagaceae'),
    'Washingtonia sp. (palma)': ('Washingtonia', 'Arecaceae'),
    "Libocedrus decurrens 'Aureovariegata'": ('Calocedro variegato', 'Cupressaceae'),
    'Pinus sylvestris': ('Pino silvestre', 'Pinaceae'),
    'Tilia x argentea': ('Tiglio argentato', 'Malvaceae'),
    'Quercus petraea': ('Farnia', 'Fagaceae'),
    'Prunus domestica subsp. insititia': ('Susino europeo', 'Rosaceae'),
    'Zyziphus jujuba': ('Giuggiolo', 'Rhamnaceae'),
    'Diospyros lotus': ('Diospiro del Caucaso', 'Ebenaceae'),
    'Mespilus germanica': ('Nespolo comune', 'Rosaceae'),
    'Prunus mahaleb': ('Ciliegio canino', 'Rosaceae'),
    'Pinus excelsa': ('Pino dell’Himalaya', 'Pinaceae'),
    'Prunus cerasus': ('Amarena / Ciliegio acido', 'Rosaceae'),
    'Cydonia oblonga': ('Cotogno', 'Rosaceae'),
    'Sorbus torminalis': ('Sorbo torminale', 'Rosaceae'),
    'Prunus serrulata': ('Ciliegio giapponese', 'Rosaceae'),
    'Acer palmatum': ('Acero palmato', 'Sapindaceae'),
    'Salix fragilis': ('Salice fragile', 'Salicaceae'),
    'Sterculia Platanifolia': ('Sterculia platanifolia', 'Malvaceae'),
    'Pyrus salicifolia': ('Pero a foglie di salice', 'Rosaceae'),
    'Acer truncatum': ('Acero troncato', 'Sapindaceae'),
    'Carya illinoensis': ('Pecan', 'Juglandaceae'),
    'Quercus cerris': ('Cerro', 'Fagaceae'),
    'Alnus cordata': ('Ontano napoletano', 'Betulaceae'),
    'Platanus orientalis': ('Platano orientale', 'Platanaceae'),
    'Maclura Pomifera': ('Maclura', 'Moraceae'),
    "Acer pseudoplatanus 'Atropurpureum'": ('Acero montano purpureo', 'Sapindaceae'),
    'Cornus sanguinea': ('Sanguinello', 'Cornaceae'),
    'Kalmia latifolia': ('Calmia', 'Ericaceae'),
    'Fagus sylvatica': ('Faggio', 'Fagaceae'),
    'Abies alba': ('Abete bianco', 'Pinaceae'),
    'Paulownia imperialis': ('Paulonia imperiale', 'Paulowniaceae'),
    'Sophora japonica pendula': ('Sofora del Giappone pendula', 'Fabaceae'),
    "Fagus sylvatica 'Purpurea'": ('Faggio rosso', 'Fagaceae'),
    'Cupressus sempervirens piramidalis': ('Cipresso sempreverde colonnare', 'Cupressaceae'),
    'Pyrus Abate': ('Pero Abate', 'Rosaceae'),
    'Pyrus Angelica': ('Pero Angelica', 'Rosaceae'),
    'Prunus amygdalus': ('Mandorlo', 'Rosaceae'),
    "Malus 'Golden Hornet'": ('Melo Golden Hornet', 'Rosaceae'),
    'Robinia bessoniana': ('Robiniera bessoniana', 'Fabaceae'),
    'Malus starking': ('Melo Starking', 'Rosaceae'),
    "Prunus armeniaca 'Portici'": ('Albicocco Portici', 'Rosaceae'),
    'Corylus avellana': ('Nocciolo comune', 'Betulaceae'),
    "Prunus armeniaca 'x Susino Biricoccolo'": ('Biricoccolo', 'Rosaceae'),
    'Sorbus domestica': ('Sorbo domestico', 'Rosaceae'),
    'Tamarix spp.': ('Tamerice', 'Tamaricaceae'),
    'Ulmus montana': ('Olmo montano', 'Ulmaceae'),
    'Trachicarpus fortunei': ('Palma di Fortune', 'Arecaceae'),
    'Malus golden': ('Melo Golden', 'Rosaceae'),
    'Sorbus x thuringiaca': ('Sorbo di Turingia', 'Rosaceae'),
    'Tilia x euchlora': ('Tiglio dell’Euchlora', 'Malvaceae'),
    'Acer rubrum': ('Acero rosso', 'Sapindaceae'),
    "Acer negundo 'Argenteo-variegatum'": ('Acero americano variegato', 'Sapindaceae'),
    'Chamaerops humilis (palma di san Pietro)': ('Palma nana', 'Arecaceae'),
    'Magnolia kobus': ('Magnolia kobus', 'Magnoliaceae'),
    'Chamaecyparis lawsoniana': ('Falsa cipressina di Lawson', 'Cupressaceae'),
    "Acer negundo 'Aureo-variegatum'": ('Acero americano aureo-variegato', 'Sapindaceae'),
    'Wisteria sinensis': ('Glicine', 'Fabaceae'),
    'Ligustrum vulgare': ('Ligustro comune', 'Oleaceae'),
    'Platanus hybrida': ('Ibrido di platano', 'Platanaceae'),
    'Thuia occidentalis': ('Tuia occidentale', 'Cupressaceae'),
    'Acer ginnala': ('Acero del fiume Amur', 'Sapindaceae'),
    'Cotinus coggygria': ('Albero della nebbia', 'Anacardiaceae'),
    "Acer platanoides 'Crimson King'": ('Acero riccio Crimson King', 'Sapindaceae'),
    'Ligustrum lucidum variegata': ('Ligustro lucido variegato', 'Oleaceae'),
    'Robinia pseudoacacia Hispida': ('Robiniera ispida', 'Fabaceae'),
    'Salix sp.': ('Salice', 'Salicaceae'),
    'Malus sylvestris': ('Melo selvatico', 'Rosaceae'),
    'Laburnum anagyroides (incluso L. alpinum)': ('Maggiociondolo', 'Fabaceae'),
    'Sorbus intermedia': ('Sorbo intermedio', 'Rosaceae'),
    "Cupressus macrocarpa 'Goldcrest'": ('Cipresso di Monterey Goldcrest', 'Cupressaceae'),
    'Chamaecyparis sspp': ('Falsa cipressina', 'Cupressaceae'),
    'Photinia serrulata': ('Photinia serrulata', 'Rosaceae'),
    'Phyllirea spp.': ('Fillirea', 'Oleaceae'),
    'Liriodendron tulipifera pyramidalis': ('Albero dei tulipani piramidale', 'Magnoliaceae'),
    'Rhamnus alaternus': ('Alaterno', 'Rhamnaceae'),
    'Amelanchier sp.': ('Amelanchier', 'Rosaceae'),
    'Tilia americana': ('Tiglio americano', 'Malvaceae'),
    'Chaenomeles japonica': ('Cotogno giapponese', 'Rosaceae'),
    'Gymnocladus dioicus': ('Albero dei fagioli del Kentucky', 'Fabaceae'),
    'Sorbus aria': ('Sorbo montano', 'Rosaceae'),
    'Eleagnus sp': ('Olivo di Boemia', 'Elaeagnaceae'),
    'Salix babylonica': ('Salice piangente', 'Salicaceae'),
    'Pinus strobus': ('Pino bianco', 'Pinaceae'),
    'Acer japonicum': ('Acero del Giappone', 'Sapindaceae'),
    'Pittosporum tobira': ('Pittosporo del Giappone', 'Pittosporaceae'),
    "Cedrus atlantica 'Glauca Pendula'": ('Cedro dell’Atlante glauco pendulo', 'Pinaceae'),
    'Castanea sativa': ('Castagno', 'Fagaceae'),
    'Pterocarya frassinifolia': ('Pterocaria', 'Juglandaceae'),
    'Quercus frainetto': ('Farnia orientale', 'Fagaceae'),
    'Cedrus libani': ('Cedro del Libano', 'Pinaceae'),
    'Pyracantha spp.': ('Pyracanta', 'Rosaceae'),
    'Acer davidii': ('Acero di David', 'Sapindaceae'),
    "Prunus fruticosa 'Globosa'": ('Ciliegio nano globoso', 'Rosaceae'),
    "Prunus armeniaca 'Tyrintos'": ('Albicocco Tyrintos', 'Rosaceae'),
    'Crataegus azarolus': ('Azzeruolo', 'Rosaceae'),
    'Cladratis lutea': ('Cladrastis del Kentucky', 'Fabaceae'),
    'Chimonanthus praecox': ('Calicanto d’inverno', 'Calycanthaceae'),
    'Buxus sempervirens': ('Bosso comune', 'Buxaceae'),
    'Acer griseum': ('Acero dalla corteccia di carta', 'Sapindaceae'),
    "Malus communis 'Calvilla bianca d'Inverno'": ('Melo Calvilla bianca d’Inverno', 'Rosaceae'),
    'Ilex aquifolium': ('Agrifoglio', 'Aquifoliaceae'),
    'Populus alba fastigiata': ('Pioppo bianco fastigiato', 'Salicaceae'),
    'Abies pinsapo': ('Abete andaluso', 'Pinaceae'),
    'Salix x chrysocoma': ('Salice piangente dorato', 'Salicaceae'),
    'Acacia dealbata': ('Mimosa', 'Fabaceae'),
    'Thuia plicata': ('Tuia gigante', 'Cupressaceae'),
    'Bignonia spp.': ('Bignonia', 'Bignoniaceae'),
    'Cornus mas': ('Corniolo', 'Cornaceae'),
    'Osmanthus aquifolium': ('Osmanto pungente', 'Oleaceae'),
    'Platanus platanor «Vallis clausa»': ('Platano Vallis Clausa', 'Platanaceae'),
    'Arbutus unedo': ('Corbezzolo', 'Ericaceae'),
    'Zelkova crenata': ('Zelkova crenata', 'Ulmaceae'),
    'Morus platanifolia': ('Gelso platanoide', 'Moraceae'),
    "Cedrus deodara 'Pendula'": ('Cedro dell’Himalaya pendulo', 'Pinaceae'),
    "Juniperus virginiana 'Skyroket'": ('Ginepro della Virginia Skyrocket', 'Cupressaceae'),
    'Betula pendula': ('Betulla verrucosa', 'Betulaceae'),
    'Forsythia x intermedia': ('Forsizia intermedia', 'Oleaceae'),
    'Pinus pinaster': ('Pino marittimo', 'Pinaceae'),
    "Cotinus coggygria 'Royal Purple'": ('Albero della nebbia Royal Purple', 'Anacardiaceae'),
    'Populus canadiensis': ('Pioppo canadese', 'Salicaceae'),
    'Salix eleagnos': ('Salice eleagno', 'Salicaceae'),
    'Pinus wallichiana': ('Pino dell’Himalaya', 'Pinaceae'),
    "Fraxinus excelsior 'Diversifolia'": ('Frassino maggiore a foglia diversa', 'Oleaceae'),
    'Prunus virginiana': ('Ciliegio della Virginia', 'Rosaceae'),
    'Abies nordmanniana': ('Abete del Caucaso', 'Pinaceae'),
    'Malus communis': ('Melo comune', 'Rosaceae'),
    'Clerodendron trichotomum': ('Clerodendro', 'Lamiaceae'),
    'Viburnum sp': ('Viburno', 'Adoxaceae'),
    'Syringa vulgaris': ('Lillà comune', 'Oleaceae'),
    'Yucca aloifolia': ('Yucca a foglie d’aloe', 'Asparagaceae'),
    'Prunus serotina': ('Ciliegio tardivo', 'Rosaceae'),
    'Quercus turneri pseudoturneri': ('Quercia di Turner', 'Fagaceae'),
    'Viburnum tinus': ('Lauro tino', 'Adoxaceae'),
    'Ligustrum jonandrum': ('Ligustro jonandrum', 'Oleaceae'),
    'Sequoia sempervirens': ('Sequoia sempreverde', 'Cupressaceae'),
    'Nerium oleander': ('Oleandro', 'Apocynaceae'),
    'Cupressus lusitanica': ('Cipresso del Portogallo', 'Cupressaceae'),
    "Prunus armeniaca 'Bella d'Imola'": ('Albicocco Bella d’Imola', 'Rosaceae'),
    "Eleagnus ebbingei 'Limelight'": ('Olivagno Limelight', 'Elaeagnaceae'),
    'Pyrus Kaiser': ('Pero Kaiser', 'Rosaceae'),
    'Rhamnus cathartica': ('Spincervino', 'Rhamnaceae'),
    "Carpinus betulus 'Purpurea'": ('Carpino rosso', 'Betulaceae'),
    "Betula pendula 'Fastigiata'": ('Betulla fastigiata', 'Betulaceae'),
    'Calycanthus praecox': ('Calicanto d’inverno', 'Calycanthaceae'),
    'Populus deltoides': ('Pioppo del Mississippi', 'Salicaceae'),
    'Pinus mugo': ('Pino mugo', 'Pinaceae'),
    'Pyrus limonia': ('Pero limonia', 'Rosaceae'),
    'Musa paradisiaca (banano)': ('Banano', 'Musaceae'),
    'Salix viminalis': ('Salice da vimini', 'Salicaceae'),
    'Vite sp': ('Vite', 'Vitaceae'),
    'Picea pungens': ('Abete del Colorado', 'Pinaceae'),
    'Quercus crenata': ('Leccio-farnia', 'Fagaceae'),
    'Deutzia x magnifica': ('Deuzia magnifica', 'Hydrangeaceae'),
    'Cercis chinensis': ('Albero di Giuda cinese', 'Fabaceae'),
    'Osmanthus sp': ('Osmanto', 'Oleaceae'),
    'Amelanchier lamarckii': ('Pero corvino', 'Rosaceae'),
    'Taxus media “Hicksii”': ('Tasso ibrido Hicksii', 'Taxaceae'),
    'Alnus incana': ('Ontano grigio', 'Betulaceae'),
    'Acer saccharum': ('Acero da zucchero', 'Sapindaceae'),
    'Rhus typhina': ('Albero della sommaco', 'Anacardiaceae'),
    'Juniperus sp.': ('Ginepro', 'Cupressaceae'),
    'Cephalotaxus fortunei': ('Cefalotasso di Fortune', 'Taxaceae'),
    'Malus Agostano': ('Melo Agostano', 'Rosaceae'),
    "Quercus hispanica 'Kewensis'": ('Quercia spagnola Kewensis', 'Fagaceae'),
    'Malus Granny Smith': ('Melo Granny Smith', 'Rosaceae'),
    "Punica granatum 'Nana'": ('Melograno nano', 'Lythraceae'),
    'Vitex agnus castus': ('Agnocasto', 'Lamiaceae'),
    'Actinidia sp.': ('Kiwi', 'Actinidiaceae')
}




In [18]:
def getfamily(s):
    r = "sconosciuto"
    try:
        if s is not None:
            r = mapping.get(s)[1]
    except Exception as e:
        pass
    return r

In [19]:
def getnomecomune(s):
    r = "sconosciuto"
    try:
        if s is not None:
            r = mapping.get(s)[0]
    except Exception as e:
        pass
    return r

In [20]:
alberi_geojson['family'] = alberi_geojson.classe.apply(lambda x: getfamily(x))

In [21]:
alberi_geojson['nome'] = alberi_geojson.classe.apply(lambda x: getnomecomune(x))

In [25]:
alberi_geojson.nome.unique()

array(['Tiglio a grandi foglie', 'Carpino fastigiato', 'Albero di Giuda',
       'Olmo', 'Farnia', 'Bagolaro', 'Tiglio ibrido', 'Platano comune',
       'Robinìa', 'Pioppo cipressino', 'Gleditsia', 'Acero americano',
       'Photinia rossa', 'Ippocastano', 'Acero campestre',
       'Cipresso di Monterey', 'Acero di monte', 'Acero riccio',
       'Olmo campestre', 'Frassino maggiore', 'Leccio',
       'Pero ornamentale', 'Tiglio selvatico', 'Gelso bianco',
       'Cedro dell’Atlante', 'Pioppo bianco',
       'Pero ornamentale Chanticlaire', 'Ciliegio dolce', 'sconosciuto',
       'Sofora del Giappone', 'Noce comune', 'Pioppo grigio',
       'Cedro dell’Himalaya', 'Pioppo nero', 'Olmo siberiano',
       'Pioppo ibrido', 'Ligustro', 'Pero domestico',
       "Cipresso dell'Arizona", 'Carpino bianco', 'Ginkgo',
       'Ippocastano rosso', 'Ciliegio ornamentale Kanzan',
       'Zelkova del Giappone', 'Acero di Freeman', 'Lagerstroemia',
       'Albero dei tulipani', 'Zelkova', 'Pioppo euroam

In [22]:
alberi_geojson.to_file("alberi.geojson", driver='GeoJSON', index=False)

In [183]:
import subprocess


geojson_file = 'alberi.geojson'
mbtiles_file =  'alberi.mbtiles'
tippecanoe = [
    "tippecanoe",
    "--force",
    "-o", mbtiles_file,
    "-l", "alberi",
    "-pC",
    "-z20",
    "-Z10",
    "-r1",
    "--no-feature-limit",
    "--no-tile-size-limit",
    geojson_file
]

result = subprocess.run(tippecanoe, capture_output=True, text=True)

In [184]:
mbtiles_file = 'alberi.mbtiles'
pmtiles_file = "alberi.pmtiles"

pmtiles = [
    "pmtiles", "convert",
    mbtiles_file,
    pmtiles_file
]

result = subprocess.run(pmtiles, capture_output=True, text=True)

In [182]:
alberi_geojson.classe.unique()

array(['Celtis australis', 'Platanus acerifolia', 'Fraxinus angustifolia',
       'Aesculus hippocastanum', 'Cedrus deodara', 'Quercus pubescens',
       'Quercus robur', 'Fraxinus excelsior',
       'Prunus cerasifera (var. Pissardi)', 'Acer campestre',
       'Cercis siliquastrum', 'Ulmus americana', 'Tilia intermedia',
       'Fraxinus intermedia', "Populus nigra 'Italica'",
       'Carpinus betulus', 'Robinia pseudoacacia', 'Pinus pinea',
       'Crataegus sspp', 'Celtis rugosa (Trema rugosa)', 'Fraxinus ornus',
       'Cedrus atlantica', 'Acer negundo', 'Tilia platyphyllos',
       'Ulmus pumila (siberiano)', 'Ulmus carpinifolia (campestre)',
       'Quercus ilex', 'Cupressus sempervirens', 'Prunus domestica',
       'Acer platanoides', 'Melia azedarach', 'Tilia cordata',
       'Ligustrum japonicum', 'Fraxinus oxycarpa', 'Ulmus sp',
       'Prunus subhirtella', 'Acer pseudoplatanus',
       'Magnolia x soulangiana', 'Morus nigra', 'Populus alba',
       'Prunus spinosa', 'Liquida